In [1]:
#This is a test page for testing web crawling
#http://titan.dcs.bbk.ac.uk/~kikpef01/testpage.html

In [2]:
#Version 1 Requirements

#As a user I want to be a ble to crawl a web page and go to pages within it for further information.

#> As a developer, I can use Python request to get a web page

#Threshold: I want to go pages that are hyperlinked
#Objective: I want to go to any page in the text even if it's not hyperlinked

#> As a developer, I can use beautifulsoup to list active hyperlinks

#Threshold: I want to crawl 2 pages deep
#Objective: I want to be able to define how far I can crawl

#> As a developer, I can use a recursive function.  I can also use a dictionary to know unique pages I already looked at.

#Threshold: I want to find keywords related to cooking
#T=0

#> As a developer, I can use regular expressions to search for common cooking related keywords.

In [1]:
startpoint = 'http://titan.dcs.bbk.ac.uk/~kikpef01/testpage.html'

In [2]:
import requests
import bs4 as bs
import re

In [3]:
r = requests.get(startpoint)

In [4]:
r.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">\n\n<HTML><HEAD><TITLE>Test Page for Crawler</TITLE>\n<META http-equiv=Content-Type content="text/html; charset=iso-8859-1">\n<META\ncontent="web crawling, test, Birkbeck, http://www.dcs.bbk.youShouldNotHaveThis</a>"\nname=keywords>\n<META content="a horrible useless page a href-ThisIsNotALink/a"\nname=description><LINK href="MyNonExistentStylesheet.css">\n<META content="MSHTML 6.00.6000.16705" name=GENERATOR></HEAD>\n<BODY text=#ff6600 bgColor=#ccffff>\n<H1 align=center>Test Page for Web Technologies Course</H1>\n<P align=left>This is some text that your crawler should ignore. This is a <A\nhref="http://www.dcs.bbk.ac.uk/~mark/" target=_blank>link</A> that you should\npick up.</P>\n<P align=left>http://www.YouShouldNotHaveThisLink.com is <FONT size=+1>a\nurl</FONT> in the text but <FONT size=+1>not</FONT> a <FONT size=+1>link</FONT>\nso you shouldn\'t get it. There may be good arguments for getting all URLs\nwhether there\

In [5]:
soup = bs.BeautifulSoup(r.text)

In [6]:
for link in soup.find_all('a'):
    print(str(link))
    m = re.search('(\'|")http.+(\'|")?', str(link), re.VERBOSE)
    if m is not None:
        print(m)
    

<a href="http://www.dcs.bbk.ac.uk/~mark/" target="_blank">link</a>
<re.Match object; span=(8, 66), match='"http://www.dcs.bbk.ac.uk/~mark/" target="_blank">
<a href="http://www.searchengineworld.com/robots/robots_tutorial.htm">robots.txt</a>
<re.Match object; span=(8, 84), match='"http://www.searchengineworld.com/robots/robots_t>
<a href="http://www.cse.ucsc.edu/~ejw/tatum/images/tatum-crawling.jpg"><font face="Verdana, Arial, Helvetica, sans-serif" size="+1">crawling</font></a>
<re.Match object; span=(8, 151), match='"http://www.cse.ucsc.edu/~ejw/tatum/images/tatum->
<a href="http://www.dcs.bbk.ac.uk/~martin/sewn/ls3/images/GoodGoing-YouGotTheLink.jpg"><img align="middle" alt="OOPS!" border="0" height="125" src="http://www.dcs.bbk.ac.uk/~martin/sewn/ls3/images/oops.gif" width="210"/></a>
<re.Match object; span=(8, 224), match='"http://www.dcs.bbk.ac.uk/~martin/sewn/ls3/images>
<a href="ftp://ftp.site.uottawa.ca/pub/courses/Winter/csi3310/CoreJavaBook/ch8/CrawlerApplet/CrawlerApplet.ja

In [29]:
def crawl_page(startpoint, **kwargs):
    depth = kwargs.get('depth_arg', 0)
    prefix = '-' * depth + '>'
    
    if depth > 2:
        print(prefix, 'Max depth exceeded, skipping page:', startpoint)
    else:
        print(prefix, 'Searching page:', startpoint)
        depth += 1
        
        r = requests.get(startpoint)
        
        soup = bs.BeautifulSoup(r.text)

        for tag in soup.find_all('a'):
            m = re.search("""href=(\'|")  # this matches the href
                        (https?://        # this starts the matching url
                        (.*?)/            # this matches the nostname
                        .*?)              # this ends the capture group for the url
                        (\'|")"""         
                        , str(tag), re.VERBOSE)
            if m is not None:
                hostname, url = m.group(3), m.group(2)
                
                if hostname not in pages_to_crawl:
                    pages_to_crawl[hostname] = set()
                
                print(f' adding {url} under {hostname}')
                pages_to_crawl[hostname].add(url)

pages_to_crawl = {}
crawl_page(startpoint)
            

> Searching page: http://titan.dcs.bbk.ac.uk/~kikpef01/testpage.html
 adding http://www.dcs.bbk.ac.uk/~mark/ under www.dcs.bbk.ac.uk
 adding http://www.searchengineworld.com/robots/robots_tutorial.htm under www.searchengineworld.com
 adding http://www.cse.ucsc.edu/~ejw/tatum/images/tatum-crawling.jpg under www.cse.ucsc.edu
 adding http://www.dcs.bbk.ac.uk/~martin/sewn/ls3/images/GoodGoing-YouGotTheLink.jpg under www.dcs.bbk.ac.uk


In [10]:
pages_to_crawl

{'http://www.dcs.bbk.ac.uk/~mark/': {'www.dcs.bbk.ac.uk'},
 'http://www.searchengineworld.com/robots/robots_tutorial.htm': {'www.searchengineworld.com'},
 'http://www.cse.ucsc.edu/~ejw/tatum/images/tatum-crawling.jpg': {'www.cse.ucsc.edu'},
 'http://www.dcs.bbk.ac.uk/~martin/sewn/ls3/images/GoodGoing-YouGotTheLink.jpg': {'www.dcs.bbk.ac.uk'}}

In [8]:
def crawl_page(startpoint, pages_to_crawl, pages_crawled):

    r = requests.get(startpoint)
        
    soup = bs.BeautifulSoup(r.text)

    for tag in soup.find_all('a'):
        m = re.search("""href=(\'|")  # this matches the href
                    (https?://        # this starts the matching url
                    (.*?)/            # this matches the hostname
                    .*?)              # this ends the capture group for the url
                    (\'|")"""         
                    , str(tag), re.VERBOSE)
        if m is not None:
            hostname, url = m.group(3), m.group(2)
                
            if hostname not in pages_to_crawl:
                pages_to_crawl[hostname] = set()
                
            print(f' adding {url} under {hostname}')
            
            pages_to_crawl[hostname].add(url)

            
def background_crawl(pages_to_crawl, pages_crawled):            
    for hostname in pages_to_crawl.copy():
        for url in pages_to_crawl[hostname].copy():
            #check if we crawled it at all, if not, put it into pages_crawled
            if hostname not in pages_crawled:
                pages_crawled[hostname] = set()
            #if we have crawled the url, say skip    
            if url in pages_crawled[hostname]:
                print(f'skipping {url}, already in crawled')
                pages_to_crawl[hostname].remove(url)
            # if we haven't crawled url, crawl
            elif url not in pages_crawled[hostname]:
                print(f'skipping {url} for more URLs')
                crawl_page(url,pages_to_crawl, pages_crawled)
                pages_crawled[hostname].add(url)
                

def run_crawl(startpoint):
    pages_to_crawl = {}
    pages_crawled = {}
    depth = 0
    
    crawl_page(startpoint, pages_to_crawl, pages_crawled)
    
    while depth<2:
        depth+=1
        background_crawl(pages_to_crawl, pages_crawled)
    
    

run_crawl(startpoint)           

 adding http://www.dcs.bbk.ac.uk/~mark/ under www.dcs.bbk.ac.uk
 adding http://www.searchengineworld.com/robots/robots_tutorial.htm under www.searchengineworld.com
 adding http://www.cse.ucsc.edu/~ejw/tatum/images/tatum-crawling.jpg under www.cse.ucsc.edu
 adding http://www.dcs.bbk.ac.uk/~martin/sewn/ls3/images/GoodGoing-YouGotTheLink.jpg under www.dcs.bbk.ac.uk
skipping http://www.dcs.bbk.ac.uk/~mark/ for more URLs
 adding http://www.dcs.bbk.ac.uk/research/research-groups/experimental-data-science/ under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/ under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/about-us/staff/ under www.dcs.bbk.ac.uk
 adding http://www.springeronline.com/sgw/cda/frontpage/0,10735,5-0-22-28710520-0,00.html under www.springeronline.com
 adding http://www.informatik.uni-trier.de/~ley/pers/hd/l/Levene:Mark under www.informatik.uni-trier.de
 adding http://www.computerchess.org.uk/ccrl/4040/ under www.computerchess.org.uk
 adding http://www.chessbase.com/ u

 adding http://www.dcs.bbk.ac.uk/ under www.dcs.bbk.ac.uk
 adding http://www.bbk.ac.uk/ under www.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#_top under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#education under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#research under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#work under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#public under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#daily_summary under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~mark/ under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~gmagoulas/ under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#_top under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#education under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#research under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~eldaw/#work under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.

 adding https://dblp.uni-trier.de"><img alt="dblp computer science bibliography" src="https://dblp.uni-trier.de/img/logo.320x120.png under dblp.uni-trier.de"><img alt="dblp computer science bibliography" src="https:
 adding https://google.com/search?q= under google.com
 adding https://google.com/search?q= under google.com
 adding https://scholar.google.com/scholar?q= under scholar.google.com
 adding https://academic.microsoft.com/#/search?iq=%2540%2540&amp;q= under academic.microsoft.com
 adding https://citeseerx.ist.psu.edu/search?q= under citeseerx.ist.psu.edu
 adding https://core.ac.uk/search?q=(null) under core.ac.uk
 adding https://www.semanticscholar.org/search?q= under www.semanticscholar.org
 adding http://www.dcs.bbk.ac.uk/~mark/ under www.dcs.bbk.ac.uk
 adding http://www.dcs.bbk.ac.uk/~mark/ under www.dcs.bbk.ac.uk
 adding https://dblp.uni-trier.de/pers/hb/l/Levene:Mark.html under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hb/l/Levene:Mark.html under dblp.uni-tr

 adding https://google.com/search?q=A+Discrete+Evolutionary+Model+for+Chess+Players%27+Ratings under google.com
 adding https://google.com/search?q=A+Discrete+Evolutionary+Model+for+Chess+Players%27+Ratings under google.com
 adding https://scholar.google.com/scholar?q=A+Discrete+Evolutionary+Model+for+Chess+Players%27+Ratings under scholar.google.com
 adding https://academic.microsoft.com/#/search?iq=%2540A+Discrete+Evolutionary+Model+for+Chess+Players%27+Ratings%2540&amp;q=A+Discrete+Evolutionary+Model+for+Chess+Players%27+Ratings under academic.microsoft.com
 adding https://citeseerx.ist.psu.edu/search?q=A+Discrete+Evolutionary+Model+for+Chess+Players%27+Ratings under citeseerx.ist.psu.edu
 adding https://core.ac.uk/search?q=A+Discrete+Evolutionary+Model+for+Chess+Players%27+Ratings under core.ac.uk
 adding https://www.semanticscholar.org/search?q=A+Discrete+Evolutionary+Model+for+Chess+Players%27+Ratings under www.semanticscholar.org
 adding https://twitter.com/intent/tweet?url=http

 adding https://dblp.uni-trier.de/rec/xml/journals/corr/abs-cs-0610060.xml under dblp.uni-trier.de
 adding https://google.com/search?q=Comparing+Typical+Opening+Move+Choices+Made+by+Humans+and+Chess+Engines under google.com
 adding https://google.com/search?q=Comparing+Typical+Opening+Move+Choices+Made+by+Humans+and+Chess+Engines under google.com
 adding https://scholar.google.com/scholar?q=Comparing+Typical+Opening+Move+Choices+Made+by+Humans+and+Chess+Engines under scholar.google.com
 adding https://academic.microsoft.com/#/search?iq=%2540Comparing+Typical+Opening+Move+Choices+Made+by+Humans+and+Chess+Engines%2540&amp;q=Comparing+Typical+Opening+Move+Choices+Made+by+Humans+and+Chess+Engines under academic.microsoft.com
 adding https://citeseerx.ist.psu.edu/search?q=Comparing+Typical+Opening+Move+Choices+Made+by+Humans+and+Chess+Engines under citeseerx.ist.psu.edu
 adding https://core.ac.uk/search?q=Comparing+Typical+Opening+Move+Choices+Made+by+Humans+and+Chess+Engines under core.ac.

 adding https://dblp.uni-trier.de/rec/bibtex/journals/tkde/LeveneV00 under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/rec/ris/journals/tkde/LeveneV00.ris under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/rec/nt/journals/tkde/LeveneV00.nt under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/rec/rdf/journals/tkde/LeveneV00.rdf under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/rec/xml/journals/tkde/LeveneV00.xml under dblp.uni-trier.de
 adding https://google.com/search?q=Justification+for+Inclusion+Dependency+Normal+Form. under google.com
 adding https://google.com/search?q=Justification+for+Inclusion+Dependency+Normal+Form. under google.com
 adding https://scholar.google.com/scholar?q=Justification+for+Inclusion+Dependency+Normal+Form. under scholar.google.com
 adding https://academic.microsoft.com/#/search?iq=%2540Justification+for+Inclusion+Dependency+Normal+Form.%2540&amp;q=Justification+for+Inclusion+Dependency+Normal+Form. under academic.microsoft.com
 

 adding https://dblp.uni-trier.de/pers/hg1/l/Levene:Mark under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hd/c/Chen_0008:Long under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hj15/l/Levene:Mark under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hd/c/Collopy:Ethan under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hj16/l/Levene:Mark under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hg1/l/Levene:Mark under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hd/c/Cox:Ingemar_J= under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hj17/l/Levene:Mark under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hg1/l/Levene:Mark under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hd/e/Eldaw:Muawya_Habib_Sarnoub under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hj18/l/Levene:Mark under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/pers/hg1/l/Levene:Mark under dblp.uni-trier.de
 addi

 adding https://dblp.uni-trier.de"><img alt="dblp computer science bibliography" src="https://dblp.uni-trier.de/img/logo.320x120.png under dblp.uni-trier.de"><img alt="dblp computer science bibliography" src="https:
 adding https://google.com/search?q= under google.com
 adding https://google.com/search?q= under google.com
 adding https://scholar.google.com/scholar?q= under scholar.google.com
 adding https://academic.microsoft.com/#/search?iq=%2540%2540&amp;q= under academic.microsoft.com
 adding https://citeseerx.ist.psu.edu/search?q= under citeseerx.ist.psu.edu
 adding https://core.ac.uk/search?q= under core.ac.uk
 adding https://www.semanticscholar.org/search?q= under www.semanticscholar.org
 adding https://dblp.uni-trier.de/faq/1474589 under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/faq/1474589 under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/faq/13501473 under dblp.uni-trier.de
 adding https://dblp.uni-trier.de/faq/11960371 under dblp.uni-trier.de
 adding https:/

KeyboardInterrupt: 

In [7]:
#6 Feb 2018
import logging

In [8]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [9]:
logging.warning("warn level")

In [10]:
logging.debug("debug level")

DEBUG:root:debug level


In [11]:
logging.info("info level")

INFO:root:info level


In [52]:
def crawl_page(startpoint, pages_to_crawl, pages_crawled, selectors, blacklist):

    r = requests.get(startpoint)
        
    soup = bs.BeautifulSoup(r.text)

    for tag in soup.find_all('a'):
        m = re.search("""href=(\'|")  # this matches the href
                    (https?://        # this starts the matching url
                    (.*?)/            # this matches the hostname
                    .*?)              # this ends the capture group for the url
                    (\'|")"""         
                    , str(tag), re.VERBOSE)
        if m is not None:
            hostname, url = m.group(3), m.group(2)

            #trying to blacklist certain sites
#             for blacklistsite in blacklist:
#                 if blacklistsite in hostname:
#                     logging.debug(f' {url} under {hostname} is blacklisted!')
#                     continue    
        
            blacklist_hits = [x for x in blacklist if x.lower() in hostname.lower()] 
            if blacklist_hits:
                logging.warning(f' {url} is blacklisted')
                continue
            
            if hostname not in pages_to_crawl:
                pages_to_crawl[hostname] = set()
            
            logging.debug(f' adding {url} under {hostname}')
            pages_to_crawl[hostname].add(url)

    for selector in selectors:
        n = re.findall(selector, r.text, re.IGNORECASE)
        if n is not None:
            print(f'Found {selector} in {startpoint}!')
            
def background_crawl(pages_to_crawl, pages_crawled, selectors, blacklist):            
    for hostname in pages_to_crawl.copy():
        for url in pages_to_crawl[hostname].copy():
            #check if we crawled it at all, if not, put it into pages_crawled
            if hostname not in pages_crawled:
                pages_crawled[hostname] = set()
            #if we have crawled the url, say skip    
            if url in pages_crawled[hostname]:
                logging.debug(f'skipping {url}, already in crawled')
                pages_to_crawl[hostname].remove(url)
            # if we haven't crawled url, crawl
            elif url not in pages_crawled[hostname]:
                logging.debug(f'skipping {url} for more URLs')
                crawl_page(url,pages_to_crawl, pages_crawled, selectors, blacklist)
                pages_crawled[hostname].add(url)
                

def run_crawl(startpoint):
    pages_to_crawl = {}
    pages_crawled = {}
    depth = 0
    
    selectors = ['big data','internet of things']
    
    blacklist = ['.mil','facebook.com','.gov','twitter.com']
    #initial seed
    crawl_page(startpoint, pages_to_crawl, pages_crawled, selectors, blacklist)
    
    while depth<2:
        depth+=1
        background_crawl(pages_to_crawl, pages_crawled, selectors, blacklist)
    
    

run_crawl(startpoint)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): titan.dcs.bbk.ac.uk:80
DEBUG:urllib3.connectionpool:http://titan.dcs.bbk.ac.uk:80 "GET /~kikpef01/testpage.html HTTP/1.1" 200 2635
DEBUG:root: adding http://www.dcs.bbk.ac.uk/~mark/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.searchengineworld.com/robots/robots_tutorial.htm under www.searchengineworld.com
DEBUG:root: adding http://www.cse.ucsc.edu/~ejw/tatum/images/tatum-crawling.jpg under www.cse.ucsc.edu
DEBUG:root: adding http://www.dcs.bbk.ac.uk/~martin/sewn/ls3/images/GoodGoing-YouGotTheLink.jpg under www.dcs.bbk.ac.uk
DEBUG:root:skipping http://www.dcs.bbk.ac.uk/~mark/ for more URLs
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.dcs.bbk.ac.uk:80


Found big data in http://titan.dcs.bbk.ac.uk/~kikpef01/testpage.html!
Found internet of things in http://titan.dcs.bbk.ac.uk/~kikpef01/testpage.html!


DEBUG:urllib3.connectionpool:http://www.dcs.bbk.ac.uk:80 "GET /~mark/ HTTP/1.1" 200 2576
DEBUG:root: adding http://www.dcs.bbk.ac.uk/research/research-groups/experimental-data-science/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.dcs.bbk.ac.uk/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.dcs.bbk.ac.uk/about-us/staff/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.springeronline.com/sgw/cda/frontpage/0,10735,5-0-22-28710520-0,00.html under www.springeronline.com
DEBUG:root: adding http://www.informatik.uni-trier.de/~ley/pers/hd/l/Levene:Mark under www.informatik.uni-trier.de
DEBUG:root: adding http://www.computerchess.org.uk/ccrl/4040/ under www.computerchess.org.uk
DEBUG:root: adding http://www.chessbase.com/ under www.chessbase.com
DEBUG:root: adding http://www.chess.com/news/ under www.chess.com
DEBUG:root: adding https://chess24.com/en/read/news under chess24.com
DEBUG:root: adding https://www.chessprogramming.org/ under www.chessprogramming.org
DEBUG:root:

Found big data in http://www.dcs.bbk.ac.uk/~mark/!
Found internet of things in http://www.dcs.bbk.ac.uk/~mark/!


DEBUG:urllib3.connectionpool:http://www.dcs.bbk.ac.uk:80 "GET /~martin/sewn/ls3/images/GoodGoing-YouGotTheLink.jpg HTTP/1.1" 404 249
DEBUG:root:skipping http://www.searchengineworld.com/robots/robots_tutorial.htm for more URLs
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.searchengineworld.com:80


...

DEBUG:urllib3.connectionpool:http://www.dcs.bbk.ac.uk:80 "GET /about-us/staff/ HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:http://www.dcs.bbk.ac.uk:80 "GET /about/people/ HTTP/1.1" 200 3914
DEBUG:root: adding http://www.dcs.bbk.ac.uk/intranet/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.bbk.ac.uk/ under www.bbk.ac.uk
DEBUG:root: adding http://www.dcs.bbk.ac.uk/intranet/ under www.dcs.bbk.ac.uk
DEBUG:root:skipping http://www.dcs.bbk.ac.uk/~abulhasan/ for more URLs
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.dcs.bbk.ac.uk:80


Found big data in http://www.dcs.bbk.ac.uk/about-us/staff/!
Found internet of things in http://www.dcs.bbk.ac.uk/about-us/staff/!


DEBUG:urllib3.connectionpool:http://www.dcs.bbk.ac.uk:80 "GET /~abulhasan/ HTTP/1.1" 200 3413
DEBUG:root: adding https://www.dcs.bbk.ac.uk/#content-start under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.bbk.ac.uk/ under www.bbk.ac.uk
DEBUG:root: adding https://www.dcs.bbk.ac.uk/ under www.dcs.bbk.ac.uk
DEBUG:root: adding https://www.dcs.bbk.ac.uk/# under www.dcs.bbk.ac.uk
DEBUG:root: adding https://www.dcs.bbk.ac.uk/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.dcs.bbk.ac.uk/intranet/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.dcs.bbk.ac.uk/~mark/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.dcs.bbk.ac.uk/~dweston/ under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.dcs.bbk.ac.uk/~michael/st/st.html under www.dcs.bbk.ac.uk
DEBUG:root: adding http://www.dcs.bbk.ac.uk/~michael/foc/foc.html under www.dcs.bbk.ac.uk
DEBUG:root: adding https://www.dcs.bbk.ac.uk/about-us/governance/ under www.dcs.bbk.ac.uk
DEBUG:root:skipping http://www.dcs.bbk.ac.uk/~marty

Found big data in http://www.dcs.bbk.ac.uk/~abulhasan/!
Found internet of things in http://www.dcs.bbk.ac.uk/~abulhasan/!


KeyboardInterrupt: 